# Laboratorio #9 - Ataque y defensa de modelos de Deep Learning

# Librerías

In [ ]:
import numpy as np
from art.estimators.classification import KerasClassifier
from art.attacks.extraction import CopycatCNN
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from art.estimators.classification import TensorFlowV2Classifier
from tensorflow.keras.models import clone_model
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from art.defences.postprocessor import ReverseSigmoid
from art.attacks.inference.membership_inference import MembershipInferenceBlackBoxRuleBased
from art.defences.trainer import AdversarialTrainer
from art.attacks.evasion import FastGradientMethod
from sklearn.metrics import accuracy_score

c:\Users\ealva\Documents\GitHub\SDC-Labs\envLabs8_9\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ealva\Documents\GitHub\SDC-Labs\envLabs8_9\lib\site-packages\art\estimators\certification\__init__.py:29: UserWarning: PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality
  warnings.warn("PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality")


# Primer ataque - Ataque de Extracción

Como primer ataque escogí un ataque de extracción ya que me parece que es uno de los ataques que cualquier modelo no se puede salvar, debido a que siempre habrá alguien que desee apropiarse del esfuerzo de otro, invirtiendo menos tiempo y dinero.
Lo que se busca es replicar la clasificación del modelo víctima utilizando solamente entradas y salidas de un dataset propio. 
Para lograr esto se entrenará un nuevo modelo solamente utilizando las salidas del modelo víctima, buscando imitar el comportamiento del modelo sin caer en la necesidad de un entrenamiento robusto pero buscando tener la información suficiente como para hacer que el nuevo modelo cuente con los pesos aproximados del modelo víctima.

## Ataque

### Cargar el modelo víctima

In [3]:
vulnerable_model = load_model('../Laboratorio8/malimg_model_saved')

### Carga de datos

In [20]:
datasetPath = "../Laboratorio8/malimg_paper_dataset_imgs"
avgHigh, avgWidth = 457, 340
batch_size = 64

datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)

attack_generator = datagen.flow_from_directory(
    datasetPath,
    target_size=(avgHigh, avgWidth),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

validation_generator = datagen.flow_from_directory(
    datasetPath,
    target_size=(avgHigh, avgWidth),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

X_list, y_list = [], []
for _ in range(40):  # 40 batches de 64 = 2560 muestras
    Xb, yb = next(attack_generator)
    X_list.append(Xb)
    y_list.append(yb)
X_attack = np.concatenate(X_list)
y_attack = np.concatenate(y_list)
X_validation, y_validation = next(validation_generator)
print(f"Datos: X:{X_attack.shape} y:{y_attack.shape}")

Found 6094 images belonging to 22 classes.
Found 1513 images belonging to 22 classes.
Datos: X:(2560, 457, 340, 1) y:(2560, 22)


### Clasificador atacante

In [21]:
loss_object = tf.keras.losses.CategoricalCrossentropy()
nb_classes = vulnerable_model.output_shape[-1]
input_shape = vulnerable_model.input_shape[1:]

clasificador_victima = TensorFlowV2Classifier(
    model=vulnerable_model,
    loss_object=loss_object,
    nb_classes=nb_classes,
    input_shape=input_shape,
    clip_values=(0, 1)
)

attacker_model = clone_model(vulnerable_model)
attacker_model.build(input_shape=(None, *input_shape))
attacker_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=loss_object,
    metrics=['accuracy']
)

clasificador_atacante = TensorFlowV2Classifier(
    model=attacker_model,
    loss_object=loss_object,
    nb_classes=nb_classes,
    input_shape=input_shape,
    clip_values=(0, 1),
    optimizer=attacker_model.optimizer,
)

In [26]:
attack = CopycatCNN(
    classifier=clasificador_victima,
    batch_size_fit=32,
    nb_epochs=20,
    nb_stolen=len(X_attack)
)

stolen_classifier = attack.extract(X_attack, y_attack, thieved_classifier=clasificador_atacante)

### Evaluación de resultados - Ataque

In [27]:
preds = stolen_classifier.predict(X_validation)
accuracy = np.mean(np.argmax(preds, axis=1) == np.argmax(y_validation, axis=1))
print(f"Precisión del modelo robado: {accuracy:.4f}")

Precisión del modelo robado: 0.7812


In [ ]:
preds_victima = clasificador_victima.predict(X_validation)
acc_victima = np.mean(np.argmax(preds_victima, axis=1) == np.argmax(y_validation, axis=1))
print(f"Precisión del modelo víctima: {acc_victima:.4f}")

preds_robado = stolen_classifier.predict(X_validation)
acc_robado = np.mean(np.argmax(preds_robado, axis=1) == np.argmax(y_validation, axis=1))
print(f"Precisión del modelo robado: {acc_robado:.4f}")

Precisión del modelo víctima: 0.7812
Precisión del modelo robado: 0.7812


In [32]:
labels_victima = np.argmax(preds_victima, axis=1)
labels_robado = np.argmax(preds_robado, axis=1)
labels_true = np.argmax(y_validation, axis=1)

for i in range(10):
    print(f"Ejemplo {i + 1}: Real={labels_true[i]}, Víctima={labels_victima[i]}, Robado={labels_robado[i]}")

coinciden = np.sum(labels_victima == labels_robado)
print(f"\nCoincidencias entre víctima y robado: {coinciden}/{len(labels_true)} ({coinciden/len(labels_true):.2%})")

Ejemplo 1: Real=1, Víctima=1, Robado=1
Ejemplo 2: Real=5, Víctima=21, Robado=21
Ejemplo 3: Real=9, Víctima=9, Robado=9
Ejemplo 4: Real=10, Víctima=10, Robado=10
Ejemplo 5: Real=15, Víctima=15, Robado=15
Ejemplo 6: Real=21, Víctima=21, Robado=21
Ejemplo 7: Real=7, Víctima=7, Robado=7
Ejemplo 8: Real=20, Víctima=20, Robado=20
Ejemplo 9: Real=20, Víctima=20, Robado=20
Ejemplo 10: Real=8, Víctima=9, Robado=9

Coincidencias entre víctima y robado: 32/32 (100.00%)


In [37]:
# Testing the performance of the original classifier
score_original = vulnerable_model.evaluate(
    x=X_validation,
    y=y_validation
    )

# Testing the performance of the stolen classifier
score_stolen = stolen_classifier.model.evaluate(
    x=X_validation, 
    y=y_validation
    )

# Comparing test losses
print(f"Original test loss: {score_original[0]:.2f} " 
      f"vs stolen test loss: {score_stolen[0]:.2f}")

# Comparing test accuracies
print(f"Original test accuracy: {score_original[1]:.2f} " 
      f"vs stolen test accuracy: {score_stolen[1]:.2f}")

1/1 [==============================] - 1s 1s/step - loss: 108.0209 - accuracy: 0.7812
Original test loss: 0.63 vs stolen test loss: 108.02
Original test accuracy: 0.78 vs stolen test accuracy: 0.78


Como se pudo observar, el ataque logró replicar muy bien el comportamiento del modelo original (vulnerable_model). Haciendo que el modelo creado a partir de él, tuviera resultados parecidos. Aunque en cuanto a la predicción de los modelos se puede ver la misma, la pérdida es muy distinta. Mientras que la pérdida del modelo original es de tan solo 0.63, la del modelo robado es de 108.02, esto se puedo haber generado por cómo se entrenó este modelo robado. Haciendo que las distribuciones de probabilidad sean más dispersas que las del modelo original.

## Defensa

### Implementación de la defensa

In [40]:
reverse_sigmoid = ReverseSigmoid(beta=3.5, apply_fit=False, apply_predict=True)
clasificador_victima.postprocessing_defences = [reverse_sigmoid]

### Clasificador atacante (ya con modelo protegido)

In [ ]:
attacker_model_protected = tf.keras.models.clone_model(vulnerable_model)
attacker_model_protected.build(input_shape=(None, *input_shape))
attacker_model_protected.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=loss_object,
    metrics=['accuracy']
)

clasificador_atacante_protegido = TensorFlowV2Classifier(
    model=attacker_model_protected,
    loss_object=loss_object,
    nb_classes=nb_classes,
    input_shape=input_shape,
    clip_values=(0, 1),
    optimizer=attacker_model_protected.optimizer,
)

attack_protected = CopycatCNN(
    batch_size_fit=32,
    nb_epochs=20,
    nb_stolen=len(X_attack),
    classifier=clasificador_victima
)

In [44]:
stolen_classifier_protected = attack_protected.extract(X_attack, y_attack, thieved_classifier=clasificador_atacante_protegido)

c:\Users\ealva\Documents\GitHub\SDC-Labs\envLabs8_9\lib\site-packages\art\defences\postprocessor\reverse_sigmoid.py:75: RuntimeWarning: divide by zero encountered in log
  perturbation_r = self.beta * (sigmoid(-self.gamma * np.log((1.0 - preds_clipped) / preds_clipped)) - 0.5)


In [49]:
# Testing the performance of the original classifier
score_original = clasificador_victima.model.evaluate(
    x=X_validation,
    y=y_validation
    )

# Testing the performance of the stolen classifier
score_stolen = stolen_classifier_protected.model.evaluate(
    x=X_validation, 
    y=y_validation
    )

# Comparing test losses
print(f"Original test loss: {score_original[0]:.2f} " 
      f"vs stolen test loss: {score_stolen[0]:.2f}")

# Comparing test accuracies
print(f"Original test accuracy: {score_original[1]:.2f} " 
      f"vs stolen test accuracy: {score_stolen[1]:.2f}")

1/1 [==============================] - 1s 997ms/step - loss: 3859.0566 - accuracy: 0.0000e+00
Original test loss: 0.63 vs stolen test loss: 3859.06
Original test accuracy: 0.78 vs stolen test accuracy: 0.00


Listo, ahora se puede apreciar como el modelo robado no logró captar el comportamiento del modelo original. Haciendo que su precisión se redujera a 0, lo cuál parece excelente ya que nos asegura que un ataque de extracción haría que el modelo resultante tuviera una precisión muy baja. Esto fue gracias a que Reverse Sigmoid distorciona las probabilidades de salida del modelo y debido a que el ataque sí o sí necesita dichos valores, su alteración hizo que los resultados del ataque se vieran muy afectados. Pues lo que busca es imitar al modelo víctima, y claro, sin la información real es imposible que algo (en cualquier contexto) pueda llegar a imitar el comportamiento. Por eso el modelo resultante del ataque imitó salidas erróneas, provocando inestabilidad en sus resultados y dando resultados nada precisos.

Además, como bien se había apreciado antes, aun sin la defensa, el modelo resultante del ataque ya contaba con una pérdida alta a comparación del original. Seguramente eso influyó en no tener la necesidad de implementar una defensa agresiva para hacer que el modelo atacante pudiera converger como lo hizo el modelo original.

# Segundo ataque - Ataque de Inferencia

## Ataque

Como segundo ataque seleccioné lo que es un ataque de inferencia. Donde buscaré determianr si una muestra específica fue utilizada como parte del dataset para el entrenamiento del modelo. Entonces como resultado espero tener un modelo el cuál sea capaz de predecir si una imagen fue o no parte del set de entrenamiento del modelo original, explotando la vulnerabilidad que tienen los modelos al mostrar una mayor "confianza" en el retorno de su clasificación con una muestra la cuál vió durante el entrenamiento.

### Cargar el modelo víctima

In [2]:
vulnerable_model_At2 = load_model('../Laboratorio8/malimg_model_saved')

### Carga de datos

In [3]:
datasetPath = "../Laboratorio8/malimg_paper_dataset_imgs"
avgHigh, avgWidth = 457, 340
batch_size = 64

datagenAt2 = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)

attack_generator_At2 = datagenAt2.flow_from_directory(
    datasetPath,
    target_size=(avgHigh, avgWidth),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

validation_generator_At2 = datagenAt2.flow_from_directory(
    datasetPath,
    target_size=(avgHigh, avgWidth),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

X_list_At2, y_list_At2 = [], []
for _ in range(40):  # 40 batches de 64 = 2560 muestras
    Xb, yb = next(attack_generator_At2)
    X_list_At2.append(Xb)
    y_list_At2.append(yb)
X_attack_At2 = np.concatenate(X_list_At2)
y_attack_At2 = np.concatenate(y_list_At2)
X_validation_At2, y_validation_At2 = next(validation_generator_At2)
print(f"Datos: X:{X_attack_At2.shape} y:{y_attack_At2.shape}")

Found 6094 images belonging to 22 classes.
Found 1513 images belonging to 22 classes.
Datos: X:(2560, 457, 340, 1) y:(2560, 22)


### Clasificador víctima

In [5]:
loss_object_At2 = tf.keras.losses.CategoricalCrossentropy()
nb_classes_At2 = vulnerable_model_At2.output_shape[-1]
input_shape_At2 = vulnerable_model_At2.input_shape[1:]

clasificador_victima_At2 = TensorFlowV2Classifier(
    model=vulnerable_model_At2,
    loss_object=loss_object_At2,
    nb_classes=nb_classes_At2,
    input_shape=input_shape_At2,
    clip_values=(0, 1),
    optimizer=vulnerable_model_At2.optimizer,
)

In [ ]:
attack_inferencia = MembershipInferenceBlackBoxRuleBased(classifier=clasificador_victima_At2)

X_mi = np.concatenate([X_attack_At2, X_validation_At2])
y_mi = np.concatenate([y_attack_At2, y_validation_At2])

y_mi_labels = np.concatenate([np.ones(len(X_attack_At2)), np.zeros(len(X_validation_At2))])

mi_pred = attack_inferencia.infer(X_mi, y_mi)

### Evaluación de resultados - Ataque

In [10]:
acc_mi = accuracy_score(y_mi_labels, mi_pred)
print(f"Precisión del ataque de inferencia de membresía: {acc_mi:.4f}")

Precisión del ataque de inferencia de membresía: 0.7735


## Defensa

### Implementación de la defensa

In [12]:
reverse_sigmoid_At2 = ReverseSigmoid(beta=3.5, apply_fit=False, apply_predict=True)
clasificador_victima_At2.postprocessing_defences = [reverse_sigmoid_At2]

attack_inferencia_def = MembershipInferenceBlackBoxRuleBased(classifier=clasificador_victima_At2)

X_mi_defended = np.concatenate([X_attack_At2, X_validation_At2])
y_mi_defended = np.concatenate([y_attack_At2, y_validation_At2])
y_mi_labels_defended = np.concatenate([np.ones(len(X_attack_At2)), np.zeros(len(X_validation_At2))])

mi_pred_def = attack_inferencia_def.infer(X_mi_defended, y_mi_defended)

c:\Users\ealva\Documents\GitHub\SDC-Labs\envLabs8_9\lib\site-packages\art\defences\postprocessor\reverse_sigmoid.py:75: RuntimeWarning: divide by zero encountered in log
  perturbation_r = self.beta * (sigmoid(-self.gamma * np.log((1.0 - preds_clipped) / preds_clipped)) - 0.5)


### Clasificador atacante (ya con modelo protegido)

In [13]:
acc_mi_def = accuracy_score(y_mi_labels_defended, mi_pred_def)
print(f"Precisión del ataque de inferencia de membresía (con defensa): {acc_mi_def:.4f}")

Precisión del ataque de inferencia de membresía (con defensa): 0.0123


Como bien se puede observar, este ataque inicialmente fue todo un éxito. El ataque mostró una precisión del 77.35%, por lo que se puede saber con gran certeza si una muestra fue o no parte del set de entrenamiento del modelo. Haciendo que el modelo sea vulnerable en este aspecto. Ya que demuestra que el modelo retorna entre su salida la confianza del modelo en sí en esa predicción, haciendo que se pueda llegar a inferir si la muestra fue o no parte del entrenamiento.
Luego, al aplicar una defensa utilizando Reverse Sigmoid, la precisión se vino en picada y cayó hasta tener solamente un 1.23% de precisión. Evidentemente la defensa cumplió su rol y ayudó que un ataque de inferencia no fuera posible con el modelo. Esto gracias a que Reverse Sigmoid distorsiona las probabilidades softmax de salida del modelo, provocando que cuando el ataque vea dichas probabilidades llegue a confundir la clasificación gracias a que la probabilidad vista por el ataque no es la real. 